# Link to model data

In [1]:
MODEL_A_GS_PATH = 'https://docs.google.com/spreadsheets/d/1KF1sLGA6D8sV6ckvpdsDrpVB9pfgPOSWW2qSzyG_zhQ/edit#gid=1989900327'
MODEL_B_GS_PATH = 'https://docs.google.com/spreadsheets/d/1Xi1gQyf7hWSjmOvsf1My08YP5qaOM55D5kTfC0E_r2o/edit#gid=1989900327'




# Support functions

In [2]:
# Support code
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

MODEL_A_GS_CSV_PATH = MODEL_A_GS_PATH.replace('/edit#gid=', '/export?format=csv&gid=')
MODEL_B_GS_CSV_PATH = MODEL_B_GS_PATH.replace('/edit#gid=', '/export?format=csv&gid=')

joint_models_df = None
unique_values = {}

all_widgets_dict = {}
out = widgets.Output()

def get_project_features(model_df): 
    model_columns = list(model_df.columns)
    return model_columns[:model_columns.index('separator')]

def get_ranking_columns(model_df):
    model_columns = list(model_df.columns)
    return model_columns[model_columns.index('separator')+1:]

def join_two_model_dfs_to_one(model_a_df, model_b_df):
    joint_models_df = pd.DataFrame()
    project_features = get_project_features(model_a_df)
    ranking_columns = get_ranking_columns(model_a_df)
    grouped_a = model_a_df.groupby(project_features)
    grouped_b = model_b_df.groupby(project_features)
    if set(grouped_a.groups.keys()) != set(grouped_b.groups.keys()):
        print("ERROR: models have different feature combinations")
        return None
    for name_of_group_from_a, group_from_model_a in grouped_a:
        group_from_model_b = grouped_b.get_group(name_of_group_from_a)
        new_df = group_from_model_a.copy()
        for ranking_column in ranking_columns:
            new_df[ranking_column +"_model_b"] = group_from_model_b[ranking_column]
        joint_models_df = pd.concat([joint_models_df, new_df], ignore_index=True)
#         display(group_from_model_a)
#         display(group_from_model_b)
#         display(joint_df)
#         return None
    
    return joint_models_df

    
    
    
def prepare_data(): 
    model_a_df = pd.read_csv(MODEL_A_GS_CSV_PATH)
    model_b_df = pd.read_csv(MODEL_B_GS_CSV_PATH)
    model_a_columns = list(model_a_df.columns)
    model_b_columns = list(model_b_df.columns)
    # Validate same columns
    if not set(model_a_columns) == set(model_b_columns):
        print("ERROR: models don't have the same features: ")
        print("Model A: " + str(set(model_a_columns)))
        print("Model B: " + str(set(model_b_columns)))
        return None
    
    project_features = get_project_features(model_a_df)

#     manufacturer_details = model_a_columns[model_a_columns.index('separator')+1:]
    for column in project_features:
        if set(model_a_df[column].unique()) != set(model_b_df[column].unique()):
            print("ERROR: column " + str(column) + " values in model A is different from model B")
            return None
        unique_values[column] = list(model_a_df[column].unique())
    # print(unique_values)
    joint_models_df = join_two_model_dfs_to_one(model_a_df, model_b_df)
    return joint_models_df, unique_values

def get_recs():
    criteria_idx = True
    for key in unique_values.keys():
#         print(str(key) + ' = ' + str(all_widgets_dict[key].value))
        criteria_idx = criteria_idx & (joint_models_df[key] == all_widgets_dict[key].value)
    for key in all_widgets_dict.keys():
        if hasattr(all_widgets_dict[key], 'value'):
            print(str(key) + ":" + str(all_widgets_dict[key].value))
    ret_df = joint_models_df[criteria_idx][['manufacturer_name', 'post_id_manuf', 'predBidProb', 'manufacturer_name_model_b', 'post_id_manuf_model_b', 'predBidProb_model_b']]
    ret_df = ret_df.rename(columns={'manufacturer_name':'A: manufacturer name', 
                            'post_id_manuf':'A: manufacturer ID', 
                           'predBidProb': 'A: predBidProb', 
                           'manufacturer_name_model_b': 'B: manufacturer name', 
                           'post_id_manuf_model_b' : 'B: manufacturer ID', 
                           'predBidProb_model_b' : 'B: predBidProb'})
    return ret_df
    
def on_get_recs_button_clicked(b):
    with out:
        clear_output()
        display(get_recs())
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Last button click: {}.".format(current_time))

def open_view():    
    all_widgets_dict = {}
    for key in unique_values.keys():   
        all_widgets_dict[key] = widgets.Dropdown(
            style={'description_width': 'initial'},
            options=unique_values[key],
            value=unique_values[key][0],
            description=key,
            disabled=False,
        )
    all_widgets_dict['get_recs_button'] = widgets.Button(description="Get recs")
    all_widgets_dict['get_recs_button'].on_click(on_get_recs_button_clicked)
    all_widgets = widgets.VBox(list(all_widgets_dict.values()))
    return all_widgets_dict, all_widgets

def show_get_recs_button(all_recs_button):
    display(out)
    display(all_recs_button)

## Read CSV 

In [3]:
joint_models_df, unique_values = prepare_data()

# Get recommendations

In [4]:
all_widgets_dict, all_widgets = open_view()
all_widgets

In [5]:
display(out)

Output()